In [ ]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [ ]:
import { TextLoader } from "langchain/document_loaders/fs/text";
const loader = new TextLoader("data/qiu.txt");
const docs = await loader.load();

In [ ]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";

const splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 500,
    chunkOverlap: 100,
  });

const splitDocs = await splitter.splitDocuments(docs);


In [ ]:
console.log(splitDocs[4])

In [ ]:
console.log(splitDocs[4].pageContent)

In [ ]:
import { OpenAIEmbeddings } from "@langchain/openai";


const embeddings = new OpenAIEmbeddings();

In [ ]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const vectorstore = new MemoryVectorStore(embeddings);
await vectorstore.addDocuments(splitDocs);

In [ ]:
const retriever = vectorstore.asRetriever(2)

In [ ]:
const res = await retriever.invoke("原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论")

In [ ]:
res

In [ ]:
import { RunnableSequence } from "@langchain/core/runnables";
import { Document } from "@langchain/core/documents";

const convertDocsToString = (documents: Document[]): string => {
     return documents.map((document) => document.pageContent).join("\n")
    }
const contextRetriverChain = RunnableSequence.from([
    (input) => input.question,
    retriever,
    convertDocsToString
])

In [ ]:
const result = await contextRetriverChain.invoke({ question: "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论"})

console.log(result)

In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const TEMPLATE = `
你是一个熟读刘慈欣的《球状闪电》的终极原著党，精通根据作品原文详细解释和回答问题，你在回答时会引用作品原文。
并且回答时仅根据原文，尽可能回答用户问题，如果原文中没有相关内容，你可以回答“原文中没有相关内容”，

以下是原文中跟用户回答相关的内容：
{context}

现在，你需要基于原文，回答以下问题：
{question}`;

const prompt = ChatPromptTemplate.fromTemplate(
    TEMPLATE
);

In [ ]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI();

In [ ]:
import { StringOutputParser } from "@langchain/core/output_parsers";

const ragChain = RunnableSequence.from([
    {
        context: contextRetriverChain,
        question: (input) => input.question,
    },
    prompt,
    model,
    new StringOutputParser()
])

In [ ]:
const answer = await ragChain.invoke({
    question: "什么是球状闪电"
  });
  
  console.log(answer);

In [ ]:
const answer = await ragChain.invoke({
    question: "详细描述原文中有什么跟直升机相关的场景"
  });

console.log(answer);